#**Evaluate iterative imputer**
**Separate a train and test set, add in our own missing values**
**Do the imputation of the training set on test set**

**Metric like MSE to assess imputation–for continuous**

**For binary–ROC curve**


#**Imputation: choose the same chemicals as in dan’s nb**
**Double check methodologies for key differences and see changes to be made to google collab** <br>
**Take logs** <br>
**Look at note under MSE root mean SE** <br>
**we could try with 5% missing,10%, 20%, etc to see how it changes** <br>
**Do the roc curve per column** <br> 
**First create a df that is t/f for what we want to make missing.**
**Make this a boolean df t/f at random** <br>
**Take actual dataset and use the t/f df to decide what will be missing** <br>
**The T/f df will be keeping track of our placeholders of missingness and make roc curves down the line easier to plot and more accurate**


# setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "thesis/collab notebooks and external code/source"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/thesis/collab notebooks and external code/source

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [3]:
!pip install xport

In [4]:
#NAIMAN'S ORIGINAL CODE
# create dictionary of data frames indexed by file names
#
import re
import os
import xport #
import pandas as pd
import numpy as np

L=os.listdir()
L=[x for x in L if re.search("_H.XPT",x)]
df={}
fnames=[]
for x in L:
    #print(x)
    with open(x,"rb") as fin:
        y=x.replace(".XPT","")
        dt=xport.to_dataframe(fin)
        if 'SEQN' in dt.columns:
            fnames.append(x.replace(".XPT",""))
            df[y]=dt
print(len(fnames))

17


In [5]:
# Dan's reduced data frame 
data=pd.read_csv('data.csv')
data.head()

,LBDPFDEL,LBXPFDE,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,LBDPFHPL,...,URXMNP,URDMNPLC,URXMOH,URDMOHLC,URXMZP,URDMZPLC,URXBPF,URDBPFLC,URXBPS,URDBPSLC
0,0.0,0.2,3.0,0.0,0.20,0.0,0.07,1.0,0.07,1.0,...,0.64,1.0,0.8,0.0,0.6,0.0,0.14,1.0,0.07,1.0
1,0.0,0.3,2.0,0.0,0.20,0.0,0.07,1.0,0.07,1.0,...,0.64,1.0,7.7,0.0,4.1,0.0,0.60,0.0,0.20,0.0
2,0.0,0.1,0.2,0.0,0.07,1.0,0.07,1.0,0.07,1.0,...,138.70,0.0,21.9,0.0,6.5,0.0,1.60,0.0,0.50,0.0
3,0.0,0.2,7.6,0.0,0.07,1.0,0.07,1.0,0.20,0.0,...,3.50,0.0,26.4,0.0,14.7,0.0,0.14,1.0,0.40,0.0
4,0.0,0.1,1.2,0.0,0.07,1.0,0.07,1.0,0.07,1.0,...,0.64,1.0,6.8,0.0,2.7,0.0,0.40,0.0,0.70,0.0


#Creating LC comment files for CUSEZN_H (Copper, Selenium & Zinc - Serum) where everything is detect for everyone

In [6]:
df['CUSEZN_H'].head()

,SEQN,WTSA2YR,LBXSCU,LBDSCUSI,LBXSSE,LBDSSESI,LBXSZN,LBDSZNSI,URXUCR
0,73560.0,183653.604036,122.0,19.15,112.2,1.42,79.9,12.22,76.0
1,73564.0,194847.483347,128.0,20.10,131.2,1.67,81.0,12.39,242.0
2,73567.0,100284.090673,128.6,20.19,114.0,1.45,73.2,11.20,215.0
3,73583.0,163017.304491,NaN,NaN,NaN,NaN,NaN,NaN,151.0
4,73585.0,55880.049721,86.1,13.52,114.5,1.45,89.8,13.74,100.0


In [7]:
df['CUSEZN_H']['LBDSCULC']=0
df['CUSEZN_H']['LBDSSELC']=0
df['CUSEZN_H']['LBDSZNLC']=0
df['CUSEZN_H']['URDUCRLC']=0

#making a dictionary of all comment codes for detection. (0 means at or above detection limit, 1 means below)

# d_p_dropna = dictionary of detect percent, omitting the Nan values

In [8]:
#df['AMDGYD_H']
#print(fnames)
d_p = {}
d_p_dropna = {}
for name in fnames:
  # for each chemical , create df with only 'LC' cols
  n = df[name].loc[:,df[name].columns.str.endswith('LC')]
  # get percentage 
  for column in n:
    p = round(((n[column][n[column]==0].count())/len(n))*100,2)
    d_p[column] = p
    p_dropna = round(((n[column][n[column]==0].count())/len(n[column].dropna()))*100,2)
    d_p_dropna[column] = p_dropna


In [9]:
#removing what is not a comment code, but just ends with 'LC'
d_p.pop("URXTLC")
d_p_dropna.pop("URXTLC")

0.0

In [10]:
d_p_dropna

{'LBD2DFLC': 18.39,
 'LBD4CELC': 0.0,
 'LBDACRLC': 100.0,
 'LBDBCDLC': 71.01,
 'LBDBGELC': 0.83,
 'LBDBGMLC': 74.49,
 'LBDBMNLC': 100.0,
 'LBDBPBLC': 100.0,
 'LBDBSELC': 100.0,
 'LBDEOALC': 96.5,
 'LBDFORLC': 100.0,
 'LBDGLYLC': 100.0,
 'LBDIHGLC': 17.39,
 'LBDPFLLC': 86.75,
 'LBDSCULC': 100.0,
 'LBDSSELC': 100.0,
 'LBDSZNLC': 100.0,
 'LBDTHGLC': 74.32,
 'LBDV06LC': 2.22,
 'LBDV07LC': 1.87,
 'LBDV08LC': 1.22,
 'LBDV1DLC': 0.0,
 'LBDV2ALC': 1.5,
 'LBDV3BLC': 0.0,
 'LBDV4CLC': 7.47,
 'LBDVBFLC': 5.48,
 'LBDVBMLC': 12.09,
 'LBDVBZLC': 24.98,
 'LBDVC6LC': 1.7,
 'LBDVCBLC': 0.03,
 'LBDVCFLC': 48.92,
 'LBDVCMLC': 10.39,
 'LBDVCTLC': 0.03,
 'LBDVDBLC': 43.76,
 'LBDVDELC': 0.0,
 'LBDVDXLC': 0.0,
 'LBDVEALC': 1.56,
 'LBDVEBLC': 27.27,
 'LBDVECLC': 0.38,
 'LBDVEELC': 0.2,
 'LBDVFNLC': 14.83,
 'LBDVFTLC': 0.0,
 'LBDVHTLC': 0.84,
 'LBDVIPLC': 0.19,
 'LBDVMCLC': 0.16,
 'LBDVMELC': 2.35,
 'LBDVMPLC': 1.69,
 'LBDVNBLC': 0.0,
 'LBDVOXLC': 26.97,
 'LBDVTCLC': 0.58,
 'LBDVTELC': 0.53,
 'LBDVTOLC': 95.81

In [11]:
len(d_p)

135

In [12]:
#np.percentile(np.array(list(d_p.items())),[0,25,50,75,100])

In [13]:
type(d_p)
L2=np.array(list(d_p.items()))

#Decide which we can treat as binary


In [27]:
# Next step: decide which variables we can treat as binary 
# fit logistic regression to predict a binary variable 

# choose maximum percent value below which the variables will be converted to binary :
  # say 30%
  
# create dictionary of variables we will switch to binary:
binary_variables = {key:val for key, val in d_p_dropna.items() if (val <= 25 or val >= 75)}

In [28]:
binary_dataset = {}

# list of chemical names 
chemicals = []
for key in binary_variables:
  chemicals.append(key)

# dictionary of columns ending in 'LC'
for key in df:
    binary_cols = df[key].loc[:,df[key].columns.str.endswith('LC')]
    binary_dataset[key] = binary_cols


In [29]:
binary_list = []
for key in binary_dataset:
  binary_dataset[key].dropna()
  binary_list.append(binary_dataset[key])


In [30]:
len(binary_list)

17

In [31]:
binary_df = pd.concat(binary_dataset.values())
binary_df.head()

,LBD2DFLC,LBD4CELC,LBDV06LC,LBDV07LC,LBDV08LC,LBDV1DLC,LBDV2ALC,LBDV3BLC,LBDV4CLC,LBDVBFLC,...,URDDEALC,URDDHDLC,LBDEOALC,LBDPFLLC,LBDSCULC,LBDSSELC,LBDSZNLC,URDUCRLC,LBDACRLC,LBDGLYLC
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
binary_df.columns

Index(['LBD2DFLC', 'LBD4CELC', 'LBDV06LC', 'LBDV07LC', 'LBDV08LC', 'LBDV1DLC',
       'LBDV2ALC', 'LBDV3BLC', 'LBDV4CLC', 'LBDVBFLC',
       ...
       'URDDEALC', 'URDDHDLC', 'LBDEOALC', 'LBDPFLLC', 'LBDSCULC', 'LBDSSELC',
       'LBDSZNLC', 'URDUCRLC', 'LBDACRLC', 'LBDGLYLC'],
      dtype='object', length=136)

In [33]:
for col in binary_df.columns:
  if col not in d_p.keys():
    binary_df.drop(col, axis=1)

# First, let's look at overlap with the PFAS Group

In [34]:
#NAIMAN'S ORIGINAL CODE
# For each pair of data frames determine the number of entries in the intersection.
# wanted to know where i have files with multiple chemicals for one person
#
ndf=len(fnames)
M=np.zeros((ndf,ndf))
for i in range(ndf):
    for j in range(ndf):
        fnamei=fnames[i]
        fnamej=fnames[j]
        dfi=df[fnamei]
        dfj=df[fnamej]
        SEQNi=set(dfi.SEQN)
        SEQNj=set(dfj.SEQN)
        M[i,j]=len(SEQNi.intersection(SEQNj))
ndf

17

In [35]:
#NAIMAN'S ORIGINAL CODE
# make list of all SEQNs in all the datasets, find that there is 9422 people
#
ALLSEQN=set()
for i in range(ndf):
    SEQN=set(df[fnames[i]].SEQN.astype(int))
    ALLSEQN=ALLSEQN.union(SEQN)
print(len(ALLSEQN))
#
# for each SEQN count number of data frames in which it appears
#
ALLSEQN=list(ALLSEQN)
print(len(ALLSEQN))
#
# index for each sequence number
#
index_of_sequence_number={}
ctr=0
for seqn in ALLSEQN:
    index_of_sequence_number[seqn]=ctr
    ctr+=1

9422
9422


In [36]:
# make list of all SEQNs in PFAS group
PFAS_SEQN=[]
for i in df['PFAS_H']['SEQN']:
  PFAS_SEQN.append(i)
len(PFAS_SEQN)

2339

In [37]:
#counting how many people show up in PFAS and other groups
count=0
for i in ALLSEQN:
  if i in PFAS_SEQN:
    count +=1
count

2339

In [38]:
#seeing which chemical groups have the same people tested as in PFAS
shared_w_PFAS=[]
for i in fnames:
  SEQN = df[i]['SEQN']
  intersection_set = set.intersection(set(SEQN), set(PFAS_SEQN))
  intersection_list = list(intersection_set)
  if len(intersection_list) > 0:
      shared_w_PFAS.append(i)
len(shared_w_PFAS)

7

In [39]:
#removing PFAS itself form this list
shared_w_PFAS.remove('PFAS_H')
shared_w_PFAS

['VOCWB_H', 'PHTHTE_H', 'EPHPP_H', 'IHGEM_H', 'PBCD_H', 'FLDEP_H']

# master dataframe

In [184]:
#making one master dataframe of everything that will be used to model PFAS_H chemicals
df_predict=pd.DataFrame(data=df['PFAS_H'])
#removing records where there are missing values in the PFAS file
df_predict=df_predict.dropna()
#the number of rows printed here should match the number of rows in the dataframe
print(df_predict.shape)
for i in shared_w_PFAS:
  df_predict=pd.merge(df_predict,pd.DataFrame(data=df[i]),on='SEQN',how='left')
df_predict

(2168, 18)


,SEQN,WTSB2YR_x,LBXPFDE,LBDPFDEL,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,...,LBDTHGSI,LBDTHGLC,LBXBSE,LBDBSESI,LBDBSELC,LBXBMN,LBDBMNSI,LBDBMNLC,LBDPFL,LBDPFLLC
0,73568.0,270133.488619,0.20,0.0,3.0,0.0,0.20,0.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73571.0,138554.659645,0.30,0.0,2.0,0.0,0.20,0.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73574.0,46186.186645,0.10,0.0,0.2,0.0,0.07,1.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73576.0,37460.999852,0.20,0.0,7.6,0.0,0.07,1.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.41,0.0
4,73577.0,128174.418941,0.10,0.0,1.2,0.0,0.07,1.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,83711.0,82946.684281,0.07,1.0,0.6,0.0,0.20,0.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2164,83712.0,114681.392602,0.20,0.0,3.2,0.0,0.07,1.0,0.07,1.0,...,1.0,1.0,169.46,2.15,0.0,5.91,107.56,0.0,NaN,NaN
2165,83713.0,49551.258249,0.90,0.0,1.1,0.0,0.07,1.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2166,83717.0,29856.202622,0.20,0.0,1.4,0.0,0.07,1.0,0.07,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
#checking that everything made it into the dataframe
count= 18
for i in shared_w_PFAS:
  j = pd.DataFrame(data=df[i])
  m = j.shape[1]-1
  count += m
count

177

#which of the chemicals do we have that we will treat as binary in our model?
#removing the continuous version of it - save this version for binary imputation purposes <br>
## df_cont_categorical: df holding all continuous variables corresponding to binary ones in df_predict

In [186]:
intersection_set = set.intersection(set(df_predict.columns), set(binary_df.columns))
intersection_list = list(intersection_set)
#removing this one because it is one of very very few chemicals that don't follow the naming convention
intersection_list.remove('URXTLC')

In [187]:
print(len(intersection_list))

73


In [188]:
#making sure we won't get rid of anything from the PFAS group that is considered a binary feature
for i in intersection_list:
  if i in df["PFAS_H"].columns:
    intersection_list.remove(i)
len(intersection_list)

73

In [189]:
intersection_list

['LBDPFLLC',
 'LBDV07LC',
 'URDMC1LC',
 'URDMEPLC',
 'URDBPSLC',
 'LBDVTPLC',
 'LBDVECLC',
 'LBDV08LC',
 'URDMHPLC',
 'URDMHHLC',
 'LBDTHGLC',
 'LBDVFNLC',
 'LBDVEELC',
 'URDBPFLC',
 'URDBP3LC',
 'LBDIHGLC',
 'URD14DLC',
 'LBDVXYLC',
 'LBDBGELC',
 'LBD4CELC',
 'LBDVTOLC',
 'URDCOPLC',
 'URDDCBLC',
 'LBDVMCLC',
 'LBDVBMLC',
 'LBDVCBLC',
 'URDCNPLC',
 'LBDBPBLC',
 'URDMOHLC',
 'URDMCHLC',
 'LBDVFTLC',
 'LBDBGMLC',
 'LBDVEBLC',
 'LBDV06LC',
 'LBDVCTLC',
 'LBDVVBLC',
 'LBDVBZLC',
 'LBDBCDLC',
 'LBDVMELC',
 'LBDVNBLC',
 'LBDVTELC',
 'LBDVHTLC',
 'LBDVBFLC',
 'URDPPBLC',
 'URDBPHLC',
 'LBDBSELC',
 'LBDVTCLC',
 'URDECPLC',
 'LBDVCFLC',
 'URDEPBLC',
 'LBDV3BLC',
 'LBDV2ALC',
 'LBDVDELC',
 'LBDVIPLC',
 'URDMIBLC',
 'URDTLCLC',
 'LBDV1DLC',
 'LBDVDXLC',
 'URDMNPLC',
 'URDMPBLC',
 'LBDBMNLC',
 'URDMBPLC',
 'LBDVDBLC',
 'LBD2DFLC',
 'LBDVMPLC',
 'LBDVC6LC',
 'LBDVOXLC',
 'URDBUPLC',
 'LBDVEALC',
 'URDTRSLC',
 'LBDV4CLC',
 'LBDVCMLC',
 'URDMZPLC']

In [190]:
#percentage of what is binary in df
len(intersection_list)/len(df_predict)*100

3.367158671586716

In [191]:
#using list comprehension to change the names back to the actual chemicals
cols_to_drop = [n[:2] + 'X' + n[3:-2] for n in intersection_list]

In [192]:
#created this with the intention of removing the levels and only keeping the detect or not LC comment code since they will be treated as binary
len(cols_to_drop)

73

In [193]:
#making sure we won't get rid of anything from the PFAS group that is considered a binary feature
for i in cols_to_drop:
  if i in df["PFAS_H"].columns:
    cols_to_drop.remove(i)

In [194]:
len(cols_to_drop)
cols_to_drop

['LBXPFL',
 'LBXV07',
 'URXMC1',
 'URXMEP',
 'URXBPS',
 'LBXVTP',
 'LBXVEC',
 'LBXV08',
 'URXMHP',
 'URXMHH',
 'LBXTHG',
 'LBXVFN',
 'LBXVEE',
 'URXBPF',
 'URXBP3',
 'LBXIHG',
 'URX14D',
 'LBXVXY',
 'LBXBGE',
 'LBX4CE',
 'LBXVTO',
 'URXCOP',
 'URXDCB',
 'LBXVMC',
 'LBXVBM',
 'LBXVCB',
 'URXCNP',
 'LBXBPB',
 'URXMOH',
 'URXMCH',
 'LBXVFT',
 'LBXBGM',
 'LBXVEB',
 'LBXV06',
 'LBXVCT',
 'LBXVVB',
 'LBXVBZ',
 'LBXBCD',
 'LBXVME',
 'LBXVNB',
 'LBXVTE',
 'LBXVHT',
 'LBXVBF',
 'URXPPB',
 'URXBPH',
 'LBXBSE',
 'LBXVTC',
 'URXECP',
 'LBXVCF',
 'URXEPB',
 'LBXV3B',
 'LBXV2A',
 'LBXVDE',
 'LBXVIP',
 'URXMIB',
 'URXTLC',
 'LBXV1D',
 'LBXVDX',
 'URXMNP',
 'URXMPB',
 'LBXBMN',
 'URXMBP',
 'LBXVDB',
 'LBX2DF',
 'LBXVMP',
 'LBXVC6',
 'LBXVOX',
 'URXBUP',
 'LBXVEA',
 'URXTRS',
 'LBXV4C',
 'LBXVCM',
 'URXMZP']

In [195]:
#there were a few with unique naming conventions to change
i=['LBXV08','LBXVHT','LBXVIP','LBXVMP','URXMCH','LBXVFT','LBXPFL','LBXV07','LBXVEE']
for j in i:
  cols_to_drop.remove(j)
i=['LBXV08N','LBXVTHF','LBXVIPB','LBXVMCP','URXMHNC','LBXVTFT','LBDPFL','LBXV07N','LBXVDEE']
for j in i:
  cols_to_drop.append(j)

In [196]:
#removing the detection levels for the chemicals in which we will be using binary detected or not in our model. (only keeping the LC detection comment code for these)
df_cont_categorical = df_predict[cols_to_drop]
df_predict=df_predict.drop(columns=cols_to_drop)

In [197]:
df_cont_categorical.head()

,URXMC1,URXMEP,URXBPS,LBXVTP,LBXVEC,URXMHP,URXMHH,LBXTHG,LBXVFN,URXBPF,...,URXMZP,LBXV08N,LBXVTHF,LBXVIPB,LBXVMCP,URXMHNC,LBXVTFT,LBDPFL,LBXV07N,LBXVDEE
0,0.28,4.2,0.07,NaN,NaN,0.57,1.2,NaN,NaN,0.14,...,0.6,NaN,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN
1,1.90,42.9,0.20,NaN,NaN,0.57,13.4,NaN,NaN,0.60,...,4.1,NaN,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN
2,143.90,9.3,0.50,NaN,NaN,7.50,49.1,NaN,NaN,1.60,...,6.5,NaN,NaN,NaN,NaN,1.40,NaN,NaN,NaN,NaN
3,6.30,14.6,0.40,NaN,NaN,5.70,40.5,NaN,NaN,0.14,...,14.7,NaN,NaN,NaN,NaN,2.00,NaN,0.41,NaN,NaN
4,2.50,12958.0,0.70,NaN,NaN,2.30,12.3,NaN,NaN,0.40,...,2.7,NaN,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN


# removing subsample weights


In [198]:
for i in df_predict.columns:
  if i.startswith('WT'):
    df_predict = df_predict.drop(columns=i)

#before we drop na values and introduce our own, let's see the percentage of missing values
– this way we know how much missingness to introduce. even though it will not match structure (due to randomness) it will match amount

In [199]:
i,j=df_predict.shape
denom=i*j
print(i, j, denom)

2168 98 212464


In [200]:
numer=df_predict.isna().sum().sum()
numer

89136

In [201]:
numer/denom*100

41.95346035093004

around 42%

# drop na values strategically

In [202]:
display(df_predict)

,SEQN,LBXPFDE,LBDPFDEL,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,...,LBDBPBLC,LBDBCDSI,LBDBCDLC,LBDTHGSI,LBDTHGLC,LBDBSESI,LBDBSELC,LBDBMNSI,LBDBMNLC,LBDPFLLC
0,73568.0,0.20,0.0,3.0,0.0,0.20,0.0,0.07,1.0,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73571.0,0.30,0.0,2.0,0.0,0.20,0.0,0.07,1.0,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73574.0,0.10,0.0,0.2,0.0,0.07,1.0,0.07,1.0,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73576.0,0.20,0.0,7.6,0.0,0.07,1.0,0.07,1.0,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,73577.0,0.10,0.0,1.2,0.0,0.07,1.0,0.07,1.0,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,83711.0,0.07,1.0,0.6,0.0,0.20,0.0,0.07,1.0,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2164,83712.0,0.20,0.0,3.2,0.0,0.07,1.0,0.07,1.0,0.07,...,0.0,0.89,0.0,1.0,1.0,2.15,0.0,107.56,0.0,NaN
2165,83713.0,0.90,0.0,1.1,0.0,0.07,1.0,0.07,1.0,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2166,83717.0,0.20,0.0,1.4,0.0,0.07,1.0,0.07,1.0,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
# empty dictionary which will be filled with column name as key 
# and number of missing values in said column as value
nan_dict={}
for col in df_predict.columns:
  nan_dict[col]=(df_predict[col].isna().sum()/len(df_predict))*100


In [204]:
#display(nan_dict)

In [205]:
nan_dict_sorted=sorted(nan_dict.items(), key=lambda item: item[1], reverse=True)

In [206]:
#display(nan_dict_sorted)

let's do the cut off at 75% 

In [207]:
dropped_cols = [] 
for key in nan_dict.keys():
  if nan_dict[key]>=75:
    #df_predict=df_predict.drop(columns=key)
    dropped_cols.append(key)

In [208]:
for i in dropped_cols:
  if i in intersection_list:
    #print(i)
    intersection_list.remove(i)

#print(dropped_cols)#print(intersection_list)

In [209]:
#using list comprehension to change the names back to the actual chemicals
cols_to_drop_new = [n[:2] + 'X' + n[3:-2] for n in intersection_list]

In [210]:
#making sure we won't get rid of anything from the PFAS group that is considered a binary feature
for i in cols_to_drop_new:
  if i in df["PFAS_H"].columns:
    cols_to_drop_new.remove(i)

In [211]:
#there were a few with unique naming conventions to change
i=['LBXV08','LBXVHT','LBXVIP','LBXVMP','URXMCH','LBXVFT','LBXPFL','LBXV07','LBXVEE']
for j in i:
  if j in cols_to_drop_new:
    cols_to_drop_new.remove(j)
i=['LBXV08N','LBXVTHF','LBXVIPB','LBXVMCP','URXMHNC','LBXVTFT','LBDPFL','LBXV07N','LBXVDEE']
for j in i:
  cols_to_drop_new.append(j)

In [213]:
for i in cols_to_drop_new:
  if i in df_cont_categorical.columns:
    df_cont_categorical.drop(columns=i)

In [214]:
df_predict = df_predict.drop(columns=dropped_cols)

In [215]:
df_predict=df_predict.dropna()

In [216]:
print(df_predict.head())
print(df_cont_categorical.head())

       SEQN  LBXPFDE  LBDPFDEL  LBXPFHS  LBDPFHSL  LBXMPAH  LBDMPAHL  LBXPFBS  \
5   73584.0      0.2       0.0      0.8       0.0     0.90       0.0     0.07   
16  73633.0      0.6       0.0      1.4       0.0     0.07       1.0     0.07   
24  73678.0      0.1       0.0      3.1       0.0     0.07       1.0     0.07   
28  73694.0      0.1       0.0      1.9       0.0     0.07       1.0     0.07   
38  73724.0      0.1       0.0      2.1       0.0     1.30       0.0     0.07   

    LBDPFBSL  LBXPFHP  ...  LBDBPBSI  LBDBPBLC  LBDBCDSI  LBDBCDLC  LBDTHGSI  \
5        1.0     0.07  ...     0.030       0.0      0.62       1.0       1.0   
16       1.0     0.10  ...     0.028       0.0      0.62       1.0       1.0   
24       1.0     0.07  ...     0.079       0.0      4.89       0.0       1.0   
28       1.0     0.07  ...     0.063       0.0      2.58       0.0       3.5   
38       1.0     0.07  ...     0.034       0.0      2.40       0.0       1.0   

    LBDTHGLC  LBDBSESI  LBDBSELC

# [DO NOT RUN: CHLOE MESSED UP TASK ORDERING] Introduce our own missing values- we will call this data frame df_missing

### create data frame of boolean values to signify which values we will impose missingness on (df_missing_boolean)

In [ ]:
import random 
df_missing=df_predict.copy()

# function to return True approx 42% of time 
def reset(percent):
    return random.randrange(100) <= percent

#empty dict to hold t/f value (t if missing, f if not)
df_missing_boolean = df_missing.copy()

for i in df_missing.columns:
  for j in range(df_missing.shape[1]):
    df_missing_boolean[i][j] = reset(42)

df_missing_boolean.head()
df_predict.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,SEQN,LBXPFDE,LBDPFDEL,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,...,LBDBPBSI,LBDBPBLC,LBDBCDSI,LBDBCDLC,LBDTHGSI,LBDTHGLC,LBDBSESI,LBDBSELC,LBDBMNSI,LBDBMNLC
5,73584.0,0.2,0.0,0.8,0.0,0.90,0.0,0.07,1.0,0.07,...,0.030,0.0,0.62,1.0,1.0,1.0,2.34,0.0,252.98,0.0
16,73633.0,0.6,0.0,1.4,0.0,0.07,1.0,0.07,1.0,0.10,...,0.028,0.0,0.62,1.0,1.0,1.0,2.14,0.0,175.08,0.0
24,73678.0,0.1,0.0,3.1,0.0,0.07,1.0,0.07,1.0,0.07,...,0.079,0.0,4.89,0.0,1.0,1.0,2.34,0.0,204.93,0.0
28,73694.0,0.1,0.0,1.9,0.0,0.07,1.0,0.07,1.0,0.07,...,0.063,0.0,2.58,0.0,3.5,0.0,3.10,0.0,160.71,0.0
38,73724.0,0.1,0.0,2.1,0.0,1.30,0.0,0.07,1.0,0.07,...,0.034,0.0,2.40,0.0,1.0,1.0,3.09,0.0,106.47,0.0


In [ ]:
import random
ix = [(row, col) for row in range(df_missing.shape[0]) for col in range(df_missing.shape[1])]
for row, col in random.sample(ix, int(round(.42*len(ix)))): #.42 for 42% missing
    df_missing.iat[row, col] = np.nan

# splitting train and test set

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_predict, test_size=0.2)

In [ ]:
print(train.isna().sum().sum(), test.isna().sum().sum())

0 0


In [ ]:
display(test)

,SEQN,LBXPFDE,LBDPFDEL,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,...,LBDBPBSI,LBDBPBLC,LBDBCDSI,LBDBCDLC,LBDTHGSI,LBDTHGLC,LBDBSESI,LBDBSELC,LBDBMNSI,LBDBMNLC
963,78113.0,0.40,0.0,2.0,0.0,0.07,1.0,0.07,1.0,0.40,...,0.020,0.0,0.62,1.0,1.0,1.0,2.18,0.0,293.38,0.0
798,77337.0,0.07,1.0,0.4,0.0,0.07,1.0,0.07,1.0,0.07,...,0.031,0.0,1.60,0.0,1.7,0.0,2.78,0.0,311.77,0.0
380,75379.0,0.60,0.0,1.1,0.0,0.07,1.0,0.07,1.0,0.07,...,0.116,0.0,3.56,0.0,41.4,0.0,2.42,0.0,232.96,0.0
1955,82739.0,0.07,1.0,0.4,0.0,0.07,1.0,0.07,1.0,0.07,...,0.028,0.0,3.29,0.0,2.7,0.0,2.38,0.0,161.07,0.0
733,77011.0,0.20,0.0,2.4,0.0,0.07,1.0,0.07,1.0,0.07,...,0.028,0.0,4.18,0.0,8.9,0.0,2.34,0.0,145.42,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,81436.0,0.07,1.0,1.0,0.0,0.20,0.0,0.07,1.0,0.07,...,0.053,0.0,6.41,0.0,1.0,1.0,1.97,0.0,137.41,0.0
895,77805.0,0.30,0.0,2.5,0.0,0.20,0.0,0.07,1.0,0.07,...,0.029,0.0,0.62,1.0,4.6,0.0,2.66,0.0,251.89,0.0
1539,80854.0,1.70,0.0,4.2,0.0,0.07,1.0,0.07,1.0,0.07,...,0.030,0.0,1.07,0.0,4.9,0.0,2.10,0.0,217.13,0.0
1960,82749.0,0.90,0.0,4.5,0.0,0.07,1.0,0.07,1.0,0.07,...,0.084,0.0,1.69,0.0,11.0,0.0,2.32,0.0,197.83,0.0


In [ ]:
display(train)

,SEQN,LBXPFDE,LBDPFDEL,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,...,LBDBPBSI,LBDBPBLC,LBDBCDSI,LBDBCDLC,LBDTHGSI,LBDTHGLC,LBDBSESI,LBDBSELC,LBDBMNSI,LBDBMNLC
681,76807.0,0.30,0.0,1.8,0.0,0.40,0.0,0.07,1.0,0.07,...,0.079,0.0,1.78,0.0,1.6,0.0,1.93,0.0,317.04,0.0
2006,82987.0,0.10,0.0,0.5,0.0,0.07,1.0,0.07,1.0,0.20,...,0.039,0.0,3.47,0.0,2.1,0.0,2.41,0.0,190.74,0.0
992,78235.0,3.60,0.0,2.3,0.0,0.07,1.0,0.07,1.0,0.07,...,0.294,0.0,4.09,0.0,231.5,0.0,2.50,0.0,219.49,0.0
1467,80490.0,0.30,0.0,0.4,0.0,0.07,1.0,0.07,1.0,0.07,...,0.054,0.0,3.47,0.0,34.8,0.0,2.82,0.0,321.59,0.0
1837,82241.0,0.20,0.0,2.3,0.0,0.20,0.0,0.07,1.0,0.07,...,0.034,0.0,4.18,0.0,22.2,0.0,4.30,0.0,107.56,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,76231.0,0.20,0.0,0.9,0.0,0.07,1.0,0.07,1.0,0.07,...,0.019,0.0,1.69,0.0,4.2,0.0,2.55,0.0,164.71,0.0
1240,79397.0,0.20,0.0,0.7,0.0,0.07,1.0,0.07,1.0,0.07,...,0.013,0.0,0.62,1.0,4.3,0.0,2.84,0.0,347.98,0.0
1570,81028.0,0.10,0.0,1.2,0.0,0.07,1.0,0.07,1.0,0.10,...,0.014,0.0,0.62,1.0,1.0,1.0,2.16,0.0,190.37,0.0
2088,83381.0,0.07,1.0,1.0,0.0,0.20,0.0,0.07,1.0,0.07,...,0.030,0.0,3.02,0.0,2.3,0.0,2.50,0.0,183.27,0.0


#make different versions for binary and continuous

In [ ]:
#filtering column names with all 0 and 1 values
train_cont= train.copy()
train_bin= train.copy()
test_cont= test.copy()
test_bin= test.copy()
bin = train.columns[train.isin([0,1]).all()]
#print(bin)
#iterating over all columns we want to use for PCA
for col in train.columns:
  #looking to see if the column is continuous for not. here we decide it is continuous if the column is not binary, and also make sure not to change SEQN
  if (col not in bin) and (col != 'SEQN'):
    #removing continuous values from binary version
    train_bin = train_bin.drop(columns=col)
  else: #removing binary values from continuous version
    train_cont = train_cont.drop(columns=col)

for col in test.columns:
  #looking to see if the column is continuous for not. here we decide it is continuous if the column is not binary, and also make sure not to change SEQN
  if (col not in bin) and (col != 'SEQN'):
    #removing continuous values from binary version
    test_bin = test_bin.drop(columns=col)
  else: #removing binary values from continuous version
    test_cont = test_cont.drop(columns=col)

In [ ]:
display(train_cont)

,LBXPFDE,LBXPFHS,LBXMPAH,LBXPFBS,LBXPFHP,LBXPFNA,LBXPFUA,LBXPFDO,URXUCR_x,URXUCR_y,LBDIHGSI,LBDBPBSI,LBDBCDSI,LBDTHGSI,LBDBSESI,LBDBMNSI
681,0.30,1.8,0.40,0.07,0.07,1.2,0.07,0.07,42.0,42.0,0.95,0.079,1.78,1.6,1.93,317.04
2006,0.10,0.5,0.07,0.07,0.20,0.4,0.07,0.07,270.0,270.0,0.95,0.039,3.47,2.1,2.41,190.74
992,3.60,2.3,0.07,0.07,0.07,10.3,2.40,0.30,81.0,81.0,7.44,0.294,4.09,231.5,2.50,219.49
1467,0.30,0.4,0.07,0.07,0.07,0.6,0.50,0.07,122.0,122.0,2.84,0.054,3.47,34.8,2.82,321.59
1837,0.20,2.3,0.20,0.07,0.07,0.7,0.30,0.07,13.0,13.0,0.95,0.034,4.18,22.2,4.30,107.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,0.20,0.9,0.07,0.07,0.07,0.6,0.10,0.07,195.0,195.0,0.95,0.019,1.69,4.2,2.55,164.71
1240,0.20,0.7,0.07,0.07,0.07,0.9,0.07,0.20,137.0,137.0,0.95,0.013,0.62,4.3,2.84,347.98
1570,0.10,1.2,0.07,0.07,0.10,0.6,0.07,0.07,127.0,127.0,0.95,0.014,0.62,1.0,2.16,190.37
2088,0.07,1.0,0.20,0.07,0.07,0.5,0.07,0.07,131.0,131.0,0.95,0.030,3.02,2.3,2.50,183.27


In [ ]:
train_bin=train_bin.drop(columns='SEQN')#SEQN is never a missing value
display(train_bin)

,LBDPFDEL,LBDPFHSL,LBDMPAHL,LBDPFBSL,LBDPFHPL,LBDPFNAL,LBDPFUAL,LBDPFDOL,LBD2DFLC,LBDV2ALC,...,URD14DLC,URDDCBLC,LBDIHGLC,LBDBGELC,LBDBGMLC,LBDBPBLC,LBDBCDLC,LBDTHGLC,LBDBSELC,LBDBMNLC
681,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2006,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
992,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1467,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1837,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1240,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1570,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2088,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
display(test_cont)

,LBXPFDE,LBXPFHS,LBXMPAH,LBXPFBS,LBXPFHP,LBXPFNA,LBXPFUA,LBXPFDO,URXUCR_x,URXUCR_y,LBDIHGSI,LBDBPBSI,LBDBCDSI,LBDTHGSI,LBDBSESI,LBDBMNSI
963,0.40,2.0,0.07,0.07,0.40,3.4,0.20,0.07,78.0,78.0,0.95,0.020,0.62,1.0,2.18,293.38
798,0.07,0.4,0.07,0.07,0.07,0.3,0.07,0.07,183.0,183.0,0.95,0.031,1.60,1.7,2.78,311.77
380,0.60,1.1,0.07,0.07,0.07,1.3,0.40,0.07,54.0,54.0,1.85,0.116,3.56,41.4,2.42,232.96
1955,0.07,0.4,0.07,0.07,0.07,0.7,0.07,0.07,65.0,65.0,0.95,0.028,3.29,2.7,2.38,161.07
733,0.20,2.4,0.07,0.07,0.07,0.6,0.07,0.07,75.0,75.0,0.95,0.028,4.18,8.9,2.34,145.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,0.07,1.0,0.20,0.07,0.07,0.4,0.07,0.07,129.0,129.0,0.95,0.053,6.41,1.0,1.97,137.41
895,0.30,2.5,0.20,0.07,0.07,1.0,0.20,0.07,16.0,16.0,0.95,0.029,0.62,4.6,2.66,251.89
1539,1.70,4.2,0.07,0.07,0.07,2.8,0.90,0.20,105.0,105.0,0.95,0.030,1.07,4.9,2.10,217.13
1960,0.90,4.5,0.07,0.07,0.07,1.6,0.60,0.07,307.0,307.0,3.74,0.084,1.69,11.0,2.32,197.83


In [ ]:
test_bin=test_bin.drop(columns='SEQN')#SEQN is never a missing value
display(test_bin)

,LBDPFDEL,LBDPFHSL,LBDMPAHL,LBDPFBSL,LBDPFHPL,LBDPFNAL,LBDPFUAL,LBDPFDOL,LBD2DFLC,LBDV2ALC,...,URD14DLC,URDDCBLC,LBDIHGLC,LBDBGELC,LBDBGMLC,LBDBPBLC,LBDBCDLC,LBDTHGLC,LBDBSELC,LBDBMNLC
963,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
798,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
380,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1955,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
733,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
895,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1539,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1960,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# checking if binary dataframes are truly binary - debugging step 
unique_test_bin = [test_bin[col].unique() for col in test_bin.columns]
unique_train_bin = [train_bin[col].unique() for col in train_bin.columns]
print(unique_test_bin, unique_train_bin, sep='\n')


[array([0., 1.]), array([0.]), array([1., 0.]), array([1.]), array([0., 1.]), array([0.]), array([0., 1.]), array([1., 0.]), array([1., 0.]), array([1., 0.]), array([1.]), array([1., 0.]), array([1.]), array([1.]), array([1.]), array([1., 0.]), array([1.]), array([1., 0.]), array([1.]), array([1.]), array([0., 1.]), array([0.]), array([0.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([1., 0.]), array([0.]), array([0.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([1., 0.]), array([0., 1.]), array([1., 0.]), array([1., 0.]), array([0.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([1., 0.]), array([1.]), array([1., 0.]), array([0.]), array([1., 0.]), array([1., 0.]), array([0.]), array([0.])]
[array([0., 1.]), array([0., 1.]), array([0., 1.]), array([1., 0.]), array([1., 0.]), array([0., 1.]), array([1., 0.]), array([1., 0.]), array([1., 0.]), array([1., 0.]), arr

#Introduce our own missing values to the training and test sets- we will call them train_missing and test_missing
–this will make comparing them for assessment easier

In [ ]:
train_cont_missing=train_cont.copy()
train_bin_missing= train_bin.copy()
test_cont_missing= test_cont.copy()
test_bin_missing= test_bin.copy()

In [ ]:
import random
def add_missing(missing_df):
  ix = [(row, col) for row in range(missing_df.shape[0]) for col in range(missing_df.shape[1])]
  for row, col in random.sample(ix, int(round(.42*len(ix)))): #.42 for 42% missing
    missing_df.iat[row, col] = np.nan

In [ ]:
add_missing(train_cont_missing)
add_missing(train_bin_missing)
add_missing(test_cont_missing)
add_missing(test_bin_missing)

In [ ]:
train_bin_missing

,LBDPFDEL,LBDPFHSL,LBDMPAHL,LBDPFBSL,LBDPFHPL,LBDPFNAL,LBDPFUAL,LBDPFDOL,LBD2DFLC,LBDV2ALC,...,URD14DLC,URDDCBLC,LBDIHGLC,LBDBGELC,LBDBGMLC,LBDBPBLC,LBDBCDLC,LBDTHGLC,LBDBSELC,LBDBMNLC
681,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,0.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0
2006,0.0,NaN,1.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,...,0.0,NaN,1.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN
992,NaN,NaN,NaN,1.0,NaN,0.0,NaN,0.0,NaN,1.0,...,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0
1467,NaN,0.0,1.0,1.0,NaN,0.0,NaN,1.0,NaN,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
1837,0.0,0.0,0.0,1.0,NaN,0.0,NaN,1.0,NaN,NaN,...,0.0,NaN,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,0.0,0.0,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,...,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0
1240,0.0,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,NaN,0.0
1570,0.0,0.0,NaN,1.0,0.0,NaN,1.0,NaN,1.0,NaN,...,0.0,0.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN
2088,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN


In [ ]:
# checking if binary dataframes are truly binary - debugging step 
unique_test_bin = [test_bin_missing[col].unique() for col in test_bin_missing.columns]
unique_train_bin = [train_bin_missing[col].unique() for col in train_bin_missing.columns]
print(unique_test_bin, unique_train_bin, sep='\n')

[array([nan,  1.,  0.]), array([ 0., nan]), array([ 1., nan,  0.]), array([ 1., nan]), array([ 0.,  1., nan]), array([nan,  0.]), array([ 0.,  1., nan]), array([ 1., nan,  0.]), array([ 1., nan,  0.]), array([ 1., nan]), array([nan,  1.]), array([ 1., nan,  0.]), array([nan,  1.]), array([ 1., nan]), array([ 1., nan]), array([ 1., nan,  0.]), array([nan,  1.]), array([ 1.,  0., nan]), array([ 1., nan]), array([nan,  1.]), array([nan,  0.]), array([ 0., nan]), array([nan,  0.]), array([ 0., nan,  1.]), array([nan,  0.,  1.]), array([ 0., nan,  1.]), array([ 0., nan]), array([nan,  1.,  0.]), array([ 0., nan,  1.]), array([nan,  0.,  1.]), array([ 1.,  0., nan]), array([nan,  0.]), array([ 0., nan]), array([nan,  0.,  1.]), array([ 0., nan,  1.]), array([ 0.,  1., nan]), array([nan,  1.,  0.]), array([ 1., nan,  0.]), array([ 0.,  1., nan]), array([ 1., nan,  0.]), array([nan,  1.,  0.]), array([ 0., nan]), array([ 0., nan]), array([nan,  0.,  1.]), array([ 0., nan,  1.]), array([nan,  1

#Multiple Imputation

continuous

In [ ]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
#from sklearn.linear_model import LogisticRegression
imp = IterativeImputer(#estimator=LogisticRegression()
    max_iter=10, random_state=0)
impfit = imp.fit(train_cont_missing)
cont_imputed = imp.transform(test_cont_missing)
cont_pred = pd.DataFrame(cont_imputed, columns = test_cont_missing.columns)
print(cont_imputed, cont_pred, sep='\n')

[[ 4.00000000e-01  2.00000000e+00  4.62012669e-01 ... -7.55878557e+00
   2.18000000e+00  2.93380000e+02]
 [ 7.00000000e-02  1.27184138e+00  1.51980692e-01 ...  1.70000000e+00
   2.49300810e+00  3.11770000e+02]
 [ 4.86255178e-01  2.00852541e+00  7.00000000e-02 ...  4.14000000e+01
   2.36088012e+00  1.72570293e+02]
 ...
 [ 2.40587105e-01  4.35643786e+00  1.02248958e-01 ...  4.90000000e+00
   2.10000000e+00  2.17130000e+02]
 [-2.15468068e+00  1.31588247e+01  7.00000000e-02 ...  1.10000000e+01
   2.32000000e+00  1.97830000e+02]
 [ 7.00000000e-02  1.00000000e+00  7.00000000e-02 ... -1.33042598e+01
   3.11000000e+00  2.71180000e+02]]
      LBXPFDE    LBXPFHS   LBXMPAH   LBXPFBS   LBXPFHP   LBXPFNA   LBXPFUA  \
0    0.400000   2.000000  0.462013  0.070000  0.400000  3.400000  0.200000   
1    0.070000   1.271841  0.151981  0.068249  0.080483  0.300000  0.070000   
2    0.486255   2.008525  0.070000  0.070000  0.045461  0.820320  0.400000   
3    0.070000   0.400000  0.070000  0.070000  0.0700

### Idea for binary: use corresponding continuous variables, impute these values, impute 1 or 0 on corresponding binary missing entries based on level of detection 

binary

In [ ]:
imp = IterativeImputer(#estimator=LogisticRegression()
    max_iter=10, random_state=0)
impfit = imp.fit(train_bin_missing)
bin_imputed = imp.transform(test_bin_missing)
bin_pred = pd.DataFrame(bin_imputed, columns = test_bin_missing.columns)
bin_pred = bin_pred.round(decimals = 0)
print(bin_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


     LBDPFDEL  LBDPFHSL  LBDMPAHL  LBDPFBSL  LBDPFHPL  LBDPFNAL  LBDPFUAL  \
0        -1.0       0.0       1.0       1.0       0.0       0.0       0.0   
1         1.0       0.0       1.0       1.0       1.0       0.0       1.0   
2        -0.0      -0.0       1.0       1.0       1.0       0.0       0.0   
3         0.0       0.0      -0.0       1.0       1.0       0.0       1.0   
4         0.0       0.0       1.0       1.0       1.0       0.0       1.0   
..        ...       ...       ...       ...       ...       ...       ...   
101       1.0       0.0       0.0       1.0       1.0       0.0       1.0   
102       0.0       0.0      -1.0       1.0       1.0       0.0       0.0   
103       0.0       0.0       1.0       1.0       1.0       0.0       0.0   
104       0.0       0.0       1.0       1.0       1.0       0.0       0.0   
105       1.0       0.0       1.0       1.0       1.0       0.0       1.0   

     LBDPFDOL  LBD2DFLC  LBDV2ALC  ...  URD14DLC  URDDCBLC  LBDIHGLC  \
0  

In [ ]:
# checking if binary dataframes are truly binary - debugging step 
unique_bin_pred = [bin_pred[col].unique() for col in bin_pred.columns]
print(unique_bin_pred)

[array([1., 0.]), array([0.]), array([1., 0.]), array([1.]), array([0., 1., 2.]), array([0.]), array([0., 1., 2.]), array([1., 0., 2.]), array([1., 0.]), array([1.]), array([1.]), array([1., 0.]), array([1., 7.]), array([1.]), array([1.]), array([1., 0.]), array([ 1., -5.]), array([1., 0.]), array([1.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0.]), array([0., 1., 2.]), array([0., 1.]), array([0., 1.]), array([1., 0., 2.]), array([ 0., -4.]), array([ 0., -4.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1., 2.]), array([1., 0.]), array([0., 1.]), array([1., 0., 2.]), array([1., 0., 2.]), array([0.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1., 2.]), array([1.]), array([0., 1.]), array([0.]), array([1., 0.]), array([0., 1.]), array([0.]), array([0.])]


#assessment
**Metric like MSE to assess imputation–for continuous**

**For binary–ROC curve**

continuous

In [ ]:
#cont_pred = pd.DataFrame(cont_imputed, columns = test_cont_missing.columns)

In [ ]:
display(cont_pred)

,LBXPFDE,LBXPFHS,LBXMPAH,LBXPFBS,LBXPFHP,LBXPFNA,LBXPFUA,LBXPFDO,URXUCR_x,URXUCR_y,LBDIHGSI,LBDBPBSI,LBDBCDSI,LBDTHGSI,LBDBSESI,LBDBMNSI
0,0.400000,2.000000,0.462013,0.070000,0.400000,3.400000,0.200000,0.066096,74.133350,78.000000,0.632658,0.020000,0.620000,-7.558786,2.180000,293.380000
1,0.070000,1.271841,0.151981,0.068249,0.080483,0.300000,0.070000,0.070000,117.108129,119.382310,1.045497,0.039110,4.357215,1.700000,2.493008,311.770000
2,0.486255,2.008525,0.070000,0.070000,0.045461,0.820320,0.400000,0.070000,54.000000,54.000000,2.458266,0.092799,0.821110,41.400000,2.360880,172.570293
3,0.070000,0.400000,0.070000,0.070000,0.070000,0.700000,0.187732,0.094384,64.866815,65.000000,0.944527,0.028000,2.647281,7.005059,2.477664,161.070000
4,0.200000,2.400000,0.216708,0.070000,0.070000,0.528480,0.070000,0.092668,75.000000,75.486425,1.494936,0.063486,4.180000,8.900000,2.340000,188.979321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,0.070000,1.000000,0.200000,0.071521,0.070000,1.291484,0.070000,0.094359,129.000000,129.000000,0.720970,0.053000,6.410000,1.000000,1.970000,137.410000
102,0.151092,2.500000,0.200000,0.070000,0.070000,0.650141,0.116658,0.070000,16.000000,16.297652,1.110393,0.037567,0.620000,4.600000,2.660000,201.440381
103,0.240587,4.356438,0.102249,0.070000,0.070000,2.800000,0.185846,0.087044,105.000000,106.257756,0.950000,0.030000,1.070000,4.900000,2.100000,217.130000
104,-2.154681,13.158825,0.070000,0.070000,0.070000,1.600000,0.600000,0.070000,307.000000,307.000000,3.740000,0.084000,1.690000,11.000000,2.320000,197.830000


In [ ]:
display(test_cont)

,LBXPFDE,LBXPFHS,LBXMPAH,LBXPFBS,LBXPFHP,LBXPFNA,LBXPFUA,LBXPFDO,URXUCR_x,URXUCR_y,LBDIHGSI,LBDBPBSI,LBDBCDSI,LBDTHGSI,LBDBSESI,LBDBMNSI
963,0.40,2.0,0.07,0.07,0.40,3.4,0.20,0.07,78.0,78.0,0.95,0.020,0.62,1.0,2.18,293.38
798,0.07,0.4,0.07,0.07,0.07,0.3,0.07,0.07,183.0,183.0,0.95,0.031,1.60,1.7,2.78,311.77
380,0.60,1.1,0.07,0.07,0.07,1.3,0.40,0.07,54.0,54.0,1.85,0.116,3.56,41.4,2.42,232.96
1955,0.07,0.4,0.07,0.07,0.07,0.7,0.07,0.07,65.0,65.0,0.95,0.028,3.29,2.7,2.38,161.07
733,0.20,2.4,0.07,0.07,0.07,0.6,0.07,0.07,75.0,75.0,0.95,0.028,4.18,8.9,2.34,145.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,0.07,1.0,0.20,0.07,0.07,0.4,0.07,0.07,129.0,129.0,0.95,0.053,6.41,1.0,1.97,137.41
895,0.30,2.5,0.20,0.07,0.07,1.0,0.20,0.07,16.0,16.0,0.95,0.029,0.62,4.6,2.66,251.89
1539,1.70,4.2,0.07,0.07,0.07,2.8,0.90,0.20,105.0,105.0,0.95,0.030,1.07,4.9,2.10,217.13
1960,0.90,4.5,0.07,0.07,0.07,1.6,0.60,0.07,307.0,307.0,3.74,0.084,1.69,11.0,2.32,197.83


In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_cont, cont_pred) 
print(mse)

155.43795777209363


In [ ]:
# setting squared=False gives RMSE
rmse = mean_squared_error(test_cont, cont_pred, squared=False)
print(rmse)

5.632712401826951


rmse* next time

what is the RMSE/mean (relative error)
  if we normalize it so it has mean 0 var 1, then when we run RMSE we are already running a relative error. Rescale all our cont var and response var after taking logs
  we could try with 5% missing,10%, 20%, etc up to 40%

binary

In [ ]:
display(test_bin)

,LBDPFDEL,LBDPFHSL,LBDMPAHL,LBDPFBSL,LBDPFHPL,LBDPFNAL,LBDPFUAL,LBDPFDOL,LBD2DFLC,LBDV2ALC,...,URD14DLC,URDDCBLC,LBDIHGLC,LBDBGELC,LBDBGMLC,LBDBPBLC,LBDBCDLC,LBDTHGLC,LBDBSELC,LBDBMNLC
963,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
798,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
380,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1955,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
733,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
895,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1539,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1960,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# have to get rid of -0.0 values for ROC curve 
for i in bin_pred.columns:
  bin_pred[i] = bin_pred[i].replace([-0.0], 0.0)
  bin_pred[i] = bin_pred[i].replace([-1.0], 1.0)

display(bin_pred)

,LBDPFDEL,LBDPFHSL,LBDMPAHL,LBDPFBSL,LBDPFHPL,LBDPFNAL,LBDPFUAL,LBDPFDOL,LBD2DFLC,LBDV2ALC,...,URD14DLC,URDDCBLC,LBDIHGLC,LBDBGELC,LBDBGMLC,LBDBPBLC,LBDBCDLC,LBDTHGLC,LBDBSELC,LBDBMNLC
0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
102,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
103,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
unique = [bin_pred[col].unique() for col in bin_pred.columns]
print(unique)

[array([1., 0.]), array([0.]), array([1., 0.]), array([1.]), array([0., 1., 2.]), array([0.]), array([0., 1., 2.]), array([1., 0., 2.]), array([1., 0.]), array([1.]), array([1.]), array([1., 0.]), array([1., 7.]), array([1.]), array([1.]), array([1., 0.]), array([ 1., -5.]), array([1., 0.]), array([1.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0.]), array([0., 1., 2.]), array([0., 1.]), array([0., 1.]), array([1., 0., 2.]), array([ 0., -4.]), array([ 0., -4.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1., 2.]), array([1., 0.]), array([0., 1.]), array([1., 0., 2.]), array([1., 0., 2.]), array([0.]), array([0., 1.]), array([0., 1.]), array([0., 1.]), array([0., 1., 2.]), array([1.]), array([0., 1.]), array([0.]), array([1., 0.]), array([0., 1.]), array([0.]), array([0.])]


## ***Issue of binary dataframe containing several non-binary values

In [ ]:
from sklearn import metrics
#ROC curve
fpr, tpr, threshold = metrics.roc_curve(test_bin, bin_pred)
auc = metrics.roc_auc_score(test_bin, bin_pred)
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

ValueError: ignored

[I have no idea what this means](https://stackoverflow.com/questions/65236646/valueerror-multilabel-indicator-format-is-not-supported-for-roc-curve-sklea)

[confused here as well](https://stackoverflow.com/questions/58430530/valueerror-multiclass-format-is-not-supported)

# Old Imputation in case we want to compare – would need to be rewritten to match training and testing structure

In [ ]:
# imputation 
# detect percentages dictionary : d_p_dropna 
import math
import random
from scipy import stats

for col in df_predict.columns:
  for row in range(0,len(df_predict)):

    # replace Nan vals with 0 or 1 based on detect percentages in d_p_dropna for binary values
    if math.isnan(df_predict[col].values[row]) and col in binary_df.columns:
      df_predict[col].values[row] = random.choices([0,1], weights = (d_p_dropna.get(col), 100.00-d_p_dropna.get(col)), k=1)[0]

    # if column is Nan but not binary replace with geometric mean in col. basing off those shared with PFAS instead of overall to be more accurate in this context
    elif math.isnan(df_predict[col].values[row]):
      df_predict[col].values[row] = stats.gmean(df_predict.dropna()[col])

In [ ]:
df_predict

,SEQN,LBXPFDE,LBDPFDEL,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,...,LBDBPBLC,LBDBCDSI,LBDBCDLC,LBDTHGSI,LBDTHGLC,LBDBSESI,LBDBSELC,LBDBMNSI,LBDBMNLC,LBDPFLLC
0,73568.0,0.20,0.0,3.0,0.0,0.20,0.0,0.07,1.0,0.07,...,0.0,1.243725,0.0,2.230941,0.0,2.454443,0.0,203.895252,0.0,0.0
1,73571.0,0.30,0.0,2.0,0.0,0.20,0.0,0.07,1.0,0.07,...,0.0,1.243725,0.0,2.230941,0.0,2.454443,0.0,203.895252,0.0,0.0
2,73574.0,0.10,0.0,0.2,0.0,0.07,1.0,0.07,1.0,0.07,...,0.0,1.243725,0.0,2.230941,0.0,2.454443,0.0,203.895252,0.0,0.0
3,73576.0,0.20,0.0,7.6,0.0,0.07,1.0,0.07,1.0,0.20,...,0.0,1.243725,0.0,2.230941,0.0,2.454443,0.0,203.895252,0.0,0.0
4,73577.0,0.10,0.0,1.2,0.0,0.07,1.0,0.07,1.0,0.07,...,0.0,1.243725,0.0,2.230941,0.0,2.454443,0.0,203.895252,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,83711.0,0.07,1.0,0.6,0.0,0.20,0.0,0.07,1.0,0.07,...,0.0,1.243725,1.0,2.230941,0.0,2.454443,0.0,203.895252,0.0,0.0
2164,83712.0,0.20,0.0,3.2,0.0,0.07,1.0,0.07,1.0,0.07,...,0.0,0.890000,0.0,1.000000,1.0,2.150000,0.0,107.560000,0.0,0.0
2165,83713.0,0.90,0.0,1.1,0.0,0.07,1.0,0.07,1.0,0.07,...,0.0,1.243725,1.0,2.230941,1.0,2.454443,0.0,203.895252,0.0,0.0
2166,83717.0,0.20,0.0,1.4,0.0,0.07,1.0,0.07,1.0,0.07,...,0.0,1.243725,0.0,2.230941,0.0,2.454443,0.0,203.895252,0.0,0.0


# Trying imputation on Dan's data frame 'data.csv' -- no missing values, can control missingness

In [ ]:
data = pd.read_csv('data.csv')
data.head()

,LBDPFDEL,LBXPFDE,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,LBDPFHPL,...,URXMNP,URDMNPLC,URXMOH,URDMOHLC,URXMZP,URDMZPLC,URXBPF,URDBPFLC,URXBPS,URDBPSLC
0,0.0,0.2,3.0,0.0,0.20,0.0,0.07,1.0,0.07,1.0,...,0.64,1.0,0.8,0.0,0.6,0.0,0.14,1.0,0.07,1.0
1,0.0,0.3,2.0,0.0,0.20,0.0,0.07,1.0,0.07,1.0,...,0.64,1.0,7.7,0.0,4.1,0.0,0.60,0.0,0.20,0.0
2,0.0,0.1,0.2,0.0,0.07,1.0,0.07,1.0,0.07,1.0,...,138.70,0.0,21.9,0.0,6.5,0.0,1.60,0.0,0.50,0.0
3,0.0,0.2,7.6,0.0,0.07,1.0,0.07,1.0,0.20,0.0,...,3.50,0.0,26.4,0.0,14.7,0.0,0.14,1.0,0.40,0.0
4,0.0,0.1,1.2,0.0,0.07,1.0,0.07,1.0,0.07,1.0,...,0.64,1.0,6.8,0.0,2.7,0.0,0.40,0.0,0.70,0.0


In [ ]:
import random 
data_missing=pd.read_csv('data.csv')

# function to return True approx 42% of time 
def reset(percent):
    return random.randrange(100) <= percent

#empty dict to hold t/f value (t if missing, f if not)
data_missing_boolean = data_missing.copy()

for i in data_missing.columns:
  for j in range(data_missing.shape[1]):
    data_missing_boolean[i][j] = reset(42)

data_missing_boolean.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,LBDPFDEL,LBXPFDE,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,LBDPFHPL,...,URXMNP,URDMNPLC,URXMOH,URDMOHLC,URXMZP,URDMZPLC,URXBPF,URDBPFLC,URXBPS,URDBPSLC
0,False,False,False,True,False,False,True,True,True,True,...,False,True,False,False,False,False,True,False,True,True
1,False,False,False,False,False,True,True,False,False,True,...,False,False,True,True,True,False,True,True,True,False
2,False,False,True,False,True,False,False,True,True,True,...,False,False,False,False,False,False,True,True,False,True
3,False,False,True,False,False,True,True,True,True,False,...,False,False,True,True,True,False,False,False,True,False
4,True,True,False,True,False,True,True,True,True,False,...,True,True,True,False,False,True,False,True,True,False


In [ ]:
for i in data_missing.columns:
  for j in range(data_missing.shape[1]):
    if data_missing_boolean[i][j]:
      data_missing[i][j] = float("Nan")

data_missing.head()

,LBDPFDEL,LBXPFDE,LBXPFHS,LBDPFHSL,LBXMPAH,LBDMPAHL,LBXPFBS,LBDPFBSL,LBXPFHP,LBDPFHPL,...,URXMNP,URDMNPLC,URXMOH,URDMOHLC,URXMZP,URDMZPLC,URXBPF,URDBPFLC,URXBPS,URDBPSLC
0,False,NaN,NaN,NaN,False,False,NaN,False,False,False,...,False,NaN,False,NaN,NaN,NaN,False,False,False,NaN
1,False,NaN,False,False,False,NaN,False,False,NaN,False,...,NaN,False,False,False,False,NaN,False,False,False,NaN
2,False,NaN,False,False,False,False,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,NaN,False,False,False,False,NaN,NaN,NaN,...,False,False,NaN,False,NaN,False,NaN,NaN,False,False
4,False,False,NaN,NaN,False,False,NaN,NaN,NaN,NaN,...,NaN,NaN,False,NaN,False,NaN,NaN,NaN,False,NaN
